In [ ]:
import json
import pandas as pd
from sklearn.ensemble import IsolationForest
import numpy as np
import random
import joblib
from datetime import datetime

COMMON_SERVICES = {
    "HTTP": 80,
    "HTTPS": 443,
    "SSH": 22,
    "WEB_API": 8080
}

def generate_realistic_network_sample():
    """Generate a realistic synthetic normal network traffic sample"""

    service_name = random.choice(list(COMMON_SERVICES.keys()))
    src_port = COMMON_SERVICES[service_name]

    if service_name == "SSH":
        protocol = "TCP"
    else:
        protocol = random.choices(["TCP", "UDP"], weights=[0.8, 0.2])[0]

    hour = datetime.now().hour
    is_night = hour < 6 or hour > 22

    if protocol == "TCP":
        packet_size = int(random.gauss(1000, 180))
    else:
        packet_size = int(random.gauss(400, 100))

    packet_size = max(100, min(1500, packet_size))

    if is_night:
        duration_ms = int(random.gauss(250, 80))
    else:
        duration_ms = int(random.gauss(120, 40))

    duration_ms = max(50, min(500, duration_ms))

    if src_port in [80, 443, 8080]:
        dst_port = random.randint(1024, 65535)  # Client-side ports
    else:  # SSH
        dst_port = random.randint(49152, 65535)  # Ephemeral client port

    return {
        "src_port": src_port,
        "dst_port": dst_port,
        "packet_size": packet_size,
        "duration_ms": duration_ms,
        "protocol": protocol
    }

# تولید داده‌های نرمال
dataset = [generate_realistic_network_sample() for _ in range(1000)]

# ذخیره داده‌ها در فایل JSON
with open("/content/training_data.json", "w") as f:
    json.dump(dataset, f, indent=2)

# بارگذاری داده‌ها
with open("/content/training_data.json") as f:
    raw_data = json.load(f)

df = pd.DataFrame(raw_data)

# پیش‌پردازش داده‌ها
def preprocess_data(df):
    df_encoded = pd.get_dummies(df, columns=["protocol"], drop_first=True)
    return df_encoded.values

preprocessed_data = preprocess_data(df)

# آموزش مدل Isolation Forest
model = IsolationForest(contamination=0.1, random_state=42, n_estimators=100)
model.fit(preprocessed_data)

# ذخیره مدل آموزش‌دیده
joblib.dump(model, "/content/anomaly_model.joblib")

# پیش‌بینی ناهنجاری برای نمونه‌ها
sample_predictions = model.predict(preprocessed_data[:10])
print("Sample predictions:", sample_predictions)
print("Legend: 1 = Normal, -1 = Anomaly")

# امتیازهای ناهنجاری
anomaly_scores = model.decision_function(preprocessed_data)
print(f"Model trained successfully!")
print(f"Training data shape: {preprocessed_data.shape}")
print(f"Average anomaly score: {np.mean(anomaly_scores):.3f}")


Sample predictions: [ 1  1  1  1  1  1  1 -1  1  1]
Legend: 1 = Normal, -1 = Anomaly
Model trained successfully!
Training data shape: (1000, 5)
Average anomaly score: 0.092


In [ ]:
!pip install pandas joblib together


In [ ]:
import joblib

# بارگذاری مدل آموزشی
model = joblib.load("/content/anomaly_model.joblib")

# بارگذاری داده‌ها (اگر قبلاً فایل‌های خود رو آپلود کرده‌ای)
import json

with open("/content/training_data.json") as f:
    raw_data = json.load(f)

import pandas as pd
df = pd.DataFrame(raw_data)

# نمایش داده‌ها برای بررسی
df.head()


,src_port,dst_port,packet_size,duration_ms,protocol
0,443,8865,828,136,TCP
1,22,57296,875,134,TCP
2,8080,19247,785,94,TCP
3,80,6573,1025,158,TCP
4,443,46887,934,118,TCP


In [ ]:
def pre_process_data(data):
    # تبدیل داده به DataFrame
    df = pd.DataFrame([data])


    df_encoded = pd.get_dummies(df, columns=["protocol"], drop_first=True)

    columns = ['src_port', 'dst_port', 'packet_size', 'duration_ms', 'protocol_UDP']
    df_encoded = df_encoded.reindex(columns=columns, fill_value=0)

    return df_encoded.values

# تست تابع پیش‌پردازش
sample_data = df.iloc[0].to_dict()  # داده نمونه برای تست
preprocessed_data = pre_process_data(sample_data)
print(preprocessed_data)


[[ 443 8865  828  136    0]]


In [ ]:
import together

# کلید API برای اتصال به LLM (اینجا باید از کلید خود استفاده کنی)
TOGETHER_API_KEY = "82557cf954a7277de823d60f8b71e6cca6a72143426cbf5323d7f5314a7de406"
together.api_key = TOGETHER_API_KEY

def get_llm_label(data):
    prompt = f"Sensor reading: {data}\nDescribe the type of anomaly and suggest a possible cause."
    response = together.ChatCompletion.create(
        model="togethercomputer/llama-3-70b-chat",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that labels sensor anomalies."},
            {"role": "user", "content": prompt}
        ],
        stream=False,
    )
    label_text = response['choices'][0]['message']['content']
    return label_text

def detect_anomaly(data):
    processed_data = pre_process_data(data)
    pred = model.predict(processed_data)[0]

    if pred == -1:  # ناهنجاری شناسایی شده
        label = get_llm_label(data)
        print(f"\n Anomaly Detected!\nLabel & Reason: {label}\n")
        return label
    else:
        print("Data is normal.")
        return None

# تست شبیه‌سازی ناهنجاری
detect_anomaly(sample_data)


Data is normal.


In [ ]:
import csv

def log_anomaly(data, label):
    with open("anomalies.csv", "a", newline='', encoding="utf-8") as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow([json.dumps(data), label])

#ذخیره کردن ناهنجاری در صورت رخداد
if detect_anomaly(sample_data):
    log_anomaly(sample_data, "Anomaly description from LLM")


Data is normal.


In [ ]:
import socket

HOST = '192.168.1.103'
PORT = 9999

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.connect((HOST, PORT))
    buffer = ""
    print("Client connected to server.\n")

    while True:
        chunk = s.recv(1024).decode()
        if not chunk:
            break
        buffer += chunk

        while '\n' in buffer:
            line, buffer = buffer.split('\n', 1)
            try:
                data = json.loads(line)
                print(f'Data Received:\n{data}\n')

                # تشخیص ناهنجاری برای هر داده
                detect_anomaly(data)

            except json.JSONDecodeError:
                print("Error decoding JSON.")


TimeoutError: [Errno 110] Connection timed out